In [27]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\mehak\AppData\Local\Temp\ipykernel_10136\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [29]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
data.fillna("",inplace=True)
print(data.shape)

(1000, 2)


### Let's divide the training and test set into two partitions

In [30]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = data['text']
y = data['label']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split
print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])


Training set size: 800
Validation set size: 200


## Data Preprocessing

In [31]:
import nltk
nltk.download('stopwords')
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mehak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [32]:
def clean_html(text):
    # Remove JavaScript/CSS
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [33]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

## Now let's work on removing stopwords
Remove the stopwords.

In [45]:
import re  # Regular expression module
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Initialize stopwords and stemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

# Text preprocessing function
def preprocess_text(text):
    text = clean_html(text)  # Remove HTML content
    text = clean_text(text)  # Clean text
    words = text.split()
    words = [stemmer.stem(word) for word in words if word not in stop_words]  # Stemming & remove stopwords
    return ' '.join(words)

# Apply preprocessing to the 'text' column of X_train and X_val
X_train['text'] = X_train['text'].apply(preprocess_text)
X_val['text'] = X_val['text'].apply(preprocess_text)

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [47]:
import re  # Regular expression module
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

# Download required NLTK packages
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')  # Ensure punkt_tab is downloaded

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_words)

# Debugging: Inspect the structure of X_train and X_val
print("Inspecting X_train and X_val structure...")
print(type(X_train))
print(X_train.head())
print(type(X_val))
print(X_val.head())

# Apply lemmatization with structure checks
if isinstance(X_train, pd.Series):
    X_train = X_train.apply(lemmatize_text)
    X_val = X_val.apply(lemmatize_text)
elif isinstance(X_train, pd.DataFrame) and 'text' in X_train.columns:
    X_train['text'] = X_train['text'].apply(lemmatize_text)
    X_val['text'] = X_val['text'].apply(lemmatize_text)
else:
    raise ValueError("X_train and X_val should either be Series or DataFrames with a 'text' column.")

# Debugging: Check transformed data
print("After transformation:")
print(X_train.head())
print(X_val.head())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mehak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mehak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mehak\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mehak\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


Inspecting X_train and X_val structure...
<class 'pandas.core.frame.DataFrame'>
                                                  text  money_mark  \
29   regard mr nelson smithkind repli privat email ...           0   
535            abl reach oscar suppo send pdb u receiv           0   
695  huma abedin bim check pat k work jack jake res...           0   
557                          announc monday cant today           0   
836  bank africaag san pedro bp san pedro cote divo...           1   

     suspicious_words  text_len  
29                  0        72  
535                 0        40  
695                 0        78  
557                 0        25  
836                 1       943  
<class 'pandas.core.frame.DataFrame'>
                                                  text  money_mark  \
521  dear sirc wish go offer consid partnerei mresa...           0   
737  take mind balkan second see ã¢ââ great plug ...           0   
740                                pls keep upd

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorizer for Bag of Words
bow_vectorizer = CountVectorizer()

# Fit and transform training data
bow_matrix = bow_vectorizer.fit_transform(X_train['text'])

# Summing up word frequencies
word_counts = bow_matrix.sum(axis=0)
words_freq = [(word, word_counts[0, idx]) for word, idx in bow_vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

# Display top 10 words
top_10_words = pd.DataFrame(words_freq[:10], columns=['Word', 'Frequency'])
print("Top 10 words in the dataset:\n", top_10_words)

Top 10 words in the dataset:
        Word  Frequency
0     money        815
1   account        720
2      bank        686
3      fund        587
4   foreign        457
5  transfer        446
6      busi        425
7   countri        399
8   contact        380
9  transact        378


## Extra features

In [64]:
print(data_train.columns)

data_train['preprocessed_text'] = data_train['text'].str.lower()  # Adjust based on your actual column name

print(data_train[['text', 'preprocessed_text']].head())

Index(['text', 'label', 'preprocessed_text'], dtype='object')
                                                text  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...   
1                                           Will do.   
2  Nora--Cheryl has emailed dozens of memos about...   
3  Dear Sir=2FMadam=2C I know that this proposal ...   
4                                                fyi   

                                   preprocessed_text  
0  dear sir, strictly a private business proposal...  
1                                           will do.  
2  nora--cheryl has emailed dozens of memos about...  
3  dear sir=2fmadam=2c i know that this proposal ...  
4                                                fyi  


In [66]:
print(data_val.columns)

data_val['preprocessed_text'] = data_val['text'].str.lower()  # Add more preprocessing if needed

print(data_val[['text', 'preprocessed_text']].head())

Index(['text'], dtype='object')
                                                text  \
0  usiness is for the fact that the deceased man ...   
1  They are happy to adjust to the afternoon. I a...   
2  Lael Brainard was confirmed 78-19 this afterno...   
3  H <hrod17@clintonemail.com>Friday March 26 201...   
4  n;"> Dear Good Friend,<br><br><br>I am happy t...   

                                   preprocessed_text  
0  usiness is for the fact that the deceased man ...  
1  they are happy to adjust to the afternoon. i a...  
2  lael brainard was confirmed 78-19 this afterno...  
3  h <hrod17@clintonemail.com>friday march 26 201...  
4  n;"> dear good friend,<br><br><br>i am happy t...  


In [67]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

# Check the resulting data
print(data_train.head())

                                                text  label  \
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1   
1                                           Will do.      0   
2  Nora--Cheryl has emailed dozens of memos about...      0   
3  Dear Sir=2FMadam=2C I know that this proposal ...      1   
4                                                fyi      0   

                                   preprocessed_text  money_mark  \
0  dear sir, strictly a private business proposal...           1   
1                                           will do.           1   
2  nora--cheryl has emailed dozens of memos about...           1   
3  dear sir=2fmadam=2c i know that this proposal ...           1   
4                                                fyi           1   

   suspicious_words  text_len  
0                 1      2292  
1                 0         8  
2                 0       197  
3                 1      2199  
4                 0         3  


In [50]:
# Apply feature engineering
X_train = add_features(X_train, 'text')
X_val = add_features(X_val, 'text')

# Preview added features
print(X_train.head())

                                                  text  money_mark  \
29   regard mr nelson smithkind repli privat email ...           0   
535            abl reach oscar suppo send pdb u receiv           0   
695  huma abedin bim check pat k work jack jake res...           0   
557                          announc monday cant today           0   
836  bank africaag san pedro bp san pedro cote divo...           1   

     suspicious_words  text_len  
29                  0        72  
535                 0        39  
695                 0        78  
557                 0        25  
836                 1       936  


## How would work the Bag of Words with Count Vectorizer concept?

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
bow_vectorizer = CountVectorizer()

# Fit and transform the training data
bow_matrix_train = bow_vectorizer.fit_transform(X_train['text'])

# Transform validation data
bow_matrix_val = bow_vectorizer.transform(X_val['text'])

# Display shape of BoW matrix
print("Shape of BoW training dataset:", bow_matrix_train.shape)
print("Shape of BoW validation dataset:", bow_matrix_val.shape)

# Display top 10 words and their counts
word_counts = bow_matrix_train.sum(axis=0)
words_freq = [(word, word_counts[0, idx]) for word, idx in bow_vectorizer.vocabulary_.items()]
words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)

# Display top 10 words
print("Top 10 words in the dataset:\n", words_freq[:10])

Shape of BoW training dataset: (800, 14941)
Shape of BoW validation dataset: (200, 14941)
Top 10 words in the dataset:
 [('money', 815), ('account', 719), ('bank', 686), ('fund', 580), ('us', 543), ('foreign', 456), ('transfer', 446), ('busi', 418), ('countri', 384), ('contact', 380)]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [41]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_val_tfidf = vectorizer.transform(X_val['text'])

# Display shape of the transformed dataset
print("Shape of TF-IDF training dataset:", X_train_tfidf.shape)
print("Shape of TF-IDF validation dataset:", X_val_tfidf.shape)

Shape of TF-IDF training dataset: (800, 14941)
Shape of TF-IDF validation dataset: (200, 14941)


## And the Train a Classifier?

In [43]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Use MultinomialNB as specified
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Predictions
y_pred = model.predict(X_val_tfidf)

# Evaluation
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

Accuracy: 0.915
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.87      0.93       125
           1       0.82      0.99      0.90        75

    accuracy                           0.92       200
   macro avg       0.91      0.93      0.91       200
weighted avg       0.93      0.92      0.92       200

Confusion Matrix:
 [[109  16]
 [  1  74]]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code